# 0 - Set up

## Install standard libraries

In [1]:
!pip3 install pyfastx
!pip3 install biopython
!pip install scikit-plot
import numpy as np
import pickle
import torch
import torch.nn as nn
import pandas as pd
import random
from tqdm import tqdm # progress bar
from Bio.Seq import Seq
from Bio.Alphabet import generic_rna
import matplotlib.pyplot as plt

In [0]:
random.seed(42)

## Import custom scripts 

### Load local files:

In [76]:
#IMPORT = True
IMPORT = False
if IMPORT == True:
  from google.colab import files
  uploaded = files.upload()

Saving preproc_pipeline.py to preproc_pipeline (1).py


In [0]:
if IMPORT == True:
  from preproc_pipeline import window_pipeline
  from warnings import simplefilter
  from get_labeled_genes import *
  from cbow_model import *

In [3]:
!ls


drive  sample_data


### Load from Google drive:

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!ls "/content/drive/My Drive"

 ari_lstm_v2.pt			 ma_rnn.pt
 cbow_kmer3_len100_epoch5.pt	'Model Outputs'
'Colab Notebooks'		 movies
 Documents			 movie_saturday2408_17.30_freepopcorn.PNG
'Documents (old stuff)'		 movie_saturday2408_17.30.PNG
 drawing_project_deep_learning	 Pictures
 Google				 sicily.gsheet
 itw.p				 sicily.xlsx
 jarubad.PNG			 wti.p
 ma_model.pt


In [10]:
!ls "/content/drive/My Drive/Model Outputs"

checkpoints  genomes  kmers  old_scripts  __pycache__  test-savings  uploads


In [11]:
!ls "/content/drive/My Drive/Model Outputs/uploads"

0_0_TEST_GCF_000008525.1_ASM852v1_genomic.fna
0_0_TEST_GCF_000008525.1_ASM852v1_genomic.fna.fxi
0_0_TRAIN_GCF_000008525.1_ASM852v1_genomic.fna
0_0_TRAIN_GCF_000008525.1_ASM852v1_genomic.fna.fxi
1_0_TEST_GCF_000008665.1_ASM866v1_genomic.fna
1_0_TEST_GCF_000008665.1_ASM866v1_genomic.fna.fxi
1_0_TRAIN_GCF_000008665.1_ASM866v1_genomic.fna
1_0_TRAIN_GCF_000008665.1_ASM866v1_genomic.fna.fxi
2_0_TEST_GCF_000008865.2_ASM886v2_genomic.fna
2_0_TEST_GCF_000008865.2_ASM886v2_genomic.fna.fxi
2_0_TRAIN_GCF_000008865.2_ASM886v2_genomic.fna
2_0_TRAIN_GCF_000008865.2_ASM886v2_genomic.fna.fxi
3_0_TEST_GCF_000009045.1_ASM904v1_genomic.fna
3_0_TEST_GCF_000009045.1_ASM904v1_genomic.fna.fxi
3_0_TRAIN_GCF_000009045.1_ASM904v1_genomic.fna
3_0_TRAIN_GCF_000009045.1_ASM904v1_genomic.fna.fxi
4_0_TEST_GCF_000027305.1_ASM2730v1_genomic.fna
4_0_TEST_GCF_000027305.1_ASM2730v1_genomic.fna.fxi
4_0_TRAIN_GCF_000027305.1_ASM2730v1_genomic.fna
4_0_TRAIN_GCF_000027305.1_ASM2730v1_genomic.fna.fxi
5_0_TEST_GCF_000027325.1_A

In [22]:
%cd "/content/drive/My Drive/Model Outputs/uploads"

/content/drive/My Drive/Model Outputs/uploads


In [0]:
from preproc_pipeline import window_pipeline
from preproc_pipeline import *
from warnings import simplefilter
from cbow_model import *

In [14]:
root = "/content/drive/My Drive/Model Outputs/uploads"
root

'/content/drive/My Drive/Model Outputs/uploads'

In [0]:
dash = '-'*80

#1 - Overview on 10 genomes 

from each genome as minimum we have:
- 1000 genes 
- 200 intergenes
- 1000 partial genes
 

 we decide to under rapresent cat 5)

5) GCF_000027325.1_ASM2732v1_genomic.fna 
- has lenght: 3116
- Number of genes: 1431
- **Number of intergenes: 264**
- Number of partial genes: 1421

for training for this genome we take 100,100(,100) genes,intergens (and partial)genes.

Looking at the second shortest:
6) GCF_000027345.1_ASM2734v1_genomic.fna 
- has lenght: 4792
- Number of genes: 2049
- **Number of intergenes: 714**
- Number of partial genes: 2029

for training for all the other genomes we take 700 genes, 700 intergenes and eventually 700 partial genes from each genome.





### Reading fasta and tsv files.

we create a dataframe and look into it

In [39]:
%cd "/content/drive/My Drive/Model Outputs/genomes"

/content/drive/My Drive/Model Outputs/genomes


In [0]:
simplefilter("ignore")
genomes = ("GCF_000008525.1_ASM852v1_genomic.fna",
"GCF_000008665.1_ASM866v1_genomic.fna",
"GCF_000008865.2_ASM886v2_genomic.fna",
"GCF_000009045.1_ASM904v1_genomic.fna",
"GCF_000027305.1_ASM2730v1_genomic.fna",
"GCF_000027325.1_ASM2732v1_genomic.fna",
"GCF_000027345.1_ASM2734v1_genomic.fna",
"GCF_000091665.1_ASM9166v1_genomic.fna",
"GCF_000214725.1_ASM21472v1_genomic.fna",
"GCF_003015225.1_ASM301522v1_genomic.fna"
)
feature_tables = ("GCA_000008525.1_ASM852v1_feature_table.tsv",
"GCA_000008665.1_ASM866v1_feature_table.tsv",
"GCA_000008865.2_ASM886v2_feature_table.tsv",
"GCA_000009045.1_ASM904v1_feature_table.tsv",
"GCA_000027305.1_ASM2730v1_feature_table.tsv",
"GCA_000027325.1_ASM2732v1_feature_table.tsv",
"GCA_000027345.1_ASM2734v1_feature_table.tsv",
"GCA_000091665.1_ASM9166v1_feature_table.tsv",
"GCA_000214725.1_ASM21472v1_feature_table.tsv",
"GCA_003015225.1_ASM301522v1_feature_table.tsv")

In [43]:
for i in range(len(genomes)):
  c = 0
  for name, seq in pyfastx.Fasta(genomes[i], build_index = False):
    #print (type(seq))
    #print (name)
    print ('%d_%d %s %s, lenght:%d'% (i,c,genomes[i],name,len(seq)))
    c = c+1
  print (dash)

  

0_0 GCF_000008525.1_ASM852v1_genomic.fna NC_000915.1, lenght:1667867
--------------------------------------------------------------------------------
1_0 GCF_000008665.1_ASM866v1_genomic.fna NC_000917.1, lenght:2178400
--------------------------------------------------------------------------------
2_0 GCF_000008865.2_ASM886v2_genomic.fna NC_002695.2, lenght:5498578
2_1 GCF_000008865.2_ASM886v2_genomic.fna NC_002128.1, lenght:92721
2_2 GCF_000008865.2_ASM886v2_genomic.fna NC_002127.1, lenght:3306
--------------------------------------------------------------------------------
3_0 GCF_000009045.1_ASM904v1_genomic.fna NC_000964.3, lenght:4215606
--------------------------------------------------------------------------------
4_0 GCF_000027305.1_ASM2730v1_genomic.fna NC_000907.1, lenght:1830138
--------------------------------------------------------------------------------
5_0 GCF_000027325.1_ASM2732v1_genomic.fna NC_000908.2, lenght:580076
-----------------------------------------------

In [59]:
gen = dict()
for i in range(len(genomes)):
  c = 0
  fna= pyfastx.Fasta(genomes[i], build_index = False)
  gen[i] = str(fna[0])
  l = len(gen[i])
  split = int(l/2)
  print ('%d_%d %s '% (i,c,genomes[i]))
  print ('%s... '%(gen[i][0:10]))
  print ('(0,%d)'%split)
  print ('(%d,%d)'%(split,l))
  print (dash)

0_0 GCF_000008525.1_ASM852v1_genomic.fna 
TGATTAGTGA... 
(0,833933)
(833933,1667867)
--------------------------------------------------------------------------------
1_0 GCF_000008665.1_ASM866v1_genomic.fna 
GAAAATGGTT... 
(0,1089200)
(1089200,2178400)
--------------------------------------------------------------------------------
2_0 GCF_000008865.2_ASM886v2_genomic.fna 
AGCTTTTCAT... 
(0,2749289)
(2749289,5498578)
--------------------------------------------------------------------------------
3_0 GCF_000009045.1_ASM904v1_genomic.fna 
ATCTTTTTCG... 
(0,2107803)
(2107803,4215606)
--------------------------------------------------------------------------------
4_0 GCF_000027305.1_ASM2730v1_genomic.fna 
TATGGCAATT... 
(0,915069)
(915069,1830138)
--------------------------------------------------------------------------------
5_0 GCF_000027325.1_ASM2732v1_genomic.fna 
TAAGTTATTA... 
(0,290038)
(290038,580076)
------------------------------------------------------------------------------

# 2 - Dataset Creation
In this first phase we ha used only the genome of E. coli.
We want to import more genome from wider range of bacteria in this second step.

## Splitting FASTA files in TRAIN and TEST

### creating TRAIN and TEST for each genome

In [0]:
twist = True
#twist = False

#### taking always first half as training and second half as test

In [0]:
if twist == False: 

  import pyfastx
  import pickle
  GEN = dict()
  splitting_loc = list()
  for i in range(len(genomes)):
    c = 0
    fna= pyfastx.Fasta(genomes[i], build_index = False)
    GEN[i] = str(fna[0])
    l = len(gen[i])
    s = int(l/2)
    print ('%d_%d %s '% (i,c,genomes[i]))
    print ('%s... tot lenght:%d '%(GEN[i][0:10],l))
    #s = int(len(GEN[i])/2)

    a = (0,s)
    b = (s,l)

    train_splits.append(a)
    test_splits.append(b)
    splitting_loc.append(s)
    
    training_fasta = GEN[i][0:s] # first half for training 
    testing_fasta = GEN[i][-s:]  # second half for testing 

    ofile = open("%s_%s_TRAIN_%s" % (i,c,genomes[i]), "w")
    ofile.write(">%s_%s_TRAIN_%s\n%s\n" % (i,c,genomes[i], training_fasta))
    ofile.close()
    print ('file %s_%s_TRAIN_%s loaded' %(i,c,genomes[i]))


    ofile = open("%s_%s_TEST_%s" % (i,c,genomes[i]), "w")
    ofile.write(">%s_%s_TEST_%s\n%s\n" % (i,c,genomes[i], testing_fasta))
    ofile.close()
    print ('file %s_%s_TEST_%s loaded' %(i,c,genomes[i]))
    print ()
    c = c+1

  print (splitting_loc)
  with open("splitting_loc.pkl", "wb") as fp:   #Pickling
    pickle.dump(splitting_loc, fp)

  print (train_splits)
  with open("train_splits.pkl", "wb") as fp:   #Pickling
    pickle.dump(train_splits, fp)

  print (test_splits)
  with open("test_splits.pkl", "wb") as fp:   #Pickling
    pickle.dump(test_splits, fp)


#### taking for training first half for even genomes

In [66]:
if twist == True: 
  import pyfastx
  import pickle
  GEN = dict()
  splitting_loc = list()
  train_splits = list()
  test_splits = list()
  for i in range(len(genomes)):
    c = 0
    fna= pyfastx.Fasta(genomes[i], build_index = False)
    GEN[i] = str(fna[0])
    l = len(gen[i])
    s = int(l/2)
    print ('%d_%d %s '% (i,c,genomes[i]))
    print ('%s... tot lenght:%d '%(GEN[i][0:10],l))
    #s = int(len(GEN[i])/2)
    splitting_loc.append(s)
    if i % 2 ==0: 
      training_fasta = GEN[i][0:s] # first half for training 
      testing_fasta = GEN[i][-s:]  # second half for testing 

      a = (0,s)
      b = (s,l)

      train_splits.append(a)
      test_splits.append(b)

      ofile = open("%s_%s_TRAIN_%s" % (i,c,genomes[i]), "w")
      ofile.write(">%s_%s_TRAIN_%s\n%s\n" % (i,c,genomes[i], training_fasta))
      ofile.close()
      print ('file %s_%s_TRAIN_%s loaded' %(i,c,genomes[i]))

      ofile = open("%s_%s_TEST_%s" % (i,c,genomes[i]), "w")
      ofile.write(">%s_%s_TEST_%s\n%s\n" % (i,c,genomes[i], testing_fasta))
      ofile.close()
      print ('file %s_%s_TEST_%s loaded' %(i,c,genomes[i]))
      print ()
    else:
      training_fasta = GEN[i][-s:]  # second half for training 
      testing_fasta = GEN[i][0:s] # first half for testing 

      a = (s,l)
      b = (0,s)

      train_splits.append(a)
      test_splits.append(b)
    
      ofile = open("%s_%s_TRAIN_%s" % (i,c,genomes[i]), "w")
      ofile.write(">%s_%s_TRAIN_%s\n%s\n" % (i,c,genomes[i], training_fasta))
      ofile.close()
      print ('file %s_%s_TRAIN_%s loaded' %(i,c,genomes[i]))

      ofile = open("%s_%s_TEST_%s" % (i,c,genomes[i]), "w")
      ofile.write(">%s_%s_TEST_%s\n%s\n" % (i,c,genomes[i], testing_fasta))
      ofile.close()
      print ('file %s_%s_TEST_%s loaded' %(i,c,genomes[i]))
      print ()
    c = c+1

  print (splitting_loc)
  with open("splitting_loc.pkl", "wb") as fp:   #Pickling
    pickle.dump(splitting_loc, fp)

  print (train_splits)
  with open("train_splits.pkl", "wb") as fp:   #Pickling
    pickle.dump(train_splits, fp)

  print (test_splits)
  with open("test_splits.pkl", "wb") as fp:   #Pickling
    pickle.dump(test_splits, fp)

0_0 GCF_000008525.1_ASM852v1_genomic.fna 
TGATTAGTGA... tot lenght:1667867 
file 0_0_TRAIN_GCF_000008525.1_ASM852v1_genomic.fna loaded
file 0_0_TEST_GCF_000008525.1_ASM852v1_genomic.fna loaded

1_0 GCF_000008665.1_ASM866v1_genomic.fna 
GAAAATGGTT... tot lenght:2178400 
file 1_0_TRAIN_GCF_000008665.1_ASM866v1_genomic.fna loaded
file 1_0_TEST_GCF_000008665.1_ASM866v1_genomic.fna loaded

2_0 GCF_000008865.2_ASM886v2_genomic.fna 
AGCTTTTCAT... tot lenght:5498578 
file 2_0_TRAIN_GCF_000008865.2_ASM886v2_genomic.fna loaded
file 2_0_TEST_GCF_000008865.2_ASM886v2_genomic.fna loaded

3_0 GCF_000009045.1_ASM904v1_genomic.fna 
ATCTTTTTCG... tot lenght:4215606 
file 3_0_TRAIN_GCF_000009045.1_ASM904v1_genomic.fna loaded
file 3_0_TEST_GCF_000009045.1_ASM904v1_genomic.fna loaded

4_0 GCF_000027305.1_ASM2730v1_genomic.fna 
TATGGCAATT... tot lenght:1830138 
file 4_0_TRAIN_GCF_000027305.1_ASM2730v1_genomic.fna loaded
file 4_0_TEST_GCF_000027305.1_ASM2730v1_genomic.fna loaded

5_0 GCF_000027325.1_ASM2732

### Loading check

In [67]:
## test

for name, seq_test in pyfastx.Fasta("0_0_TRAIN_GCF_000008525.1_ASM852v1_genomic.fna", build_index = False):
  print (name)
  print (len(seq_test), seq_test[0:10])
#print (seq_test)

0_0_TRAIN_GCF_000008525.1_ASM852v1_genomic.fna
833933 TGATTAGTGA


# 3 - Dataset Loading

## unpickling

In [0]:
!ls "/content/drive/My Drive/Model Outputs/genomes"

In [68]:
%cd "/content/drive/My Drive/Model Outputs/genomes"

with open("train_splits.pkl", "rb") as fp:   # Unpickling
  train_splits = pickle.load(fp)
print (train_splits)

with open("test_splits.pkl", "rb") as fp:   # Unpickling
  test_splits = pickle.load(fp)
print (test_splits)

/content/drive/My Drive/Model Outputs/genomes
[(0, 833933), (1089200, 2178400), (0, 2749289), (2107803, 4215606), (0, 915069), (290038, 580076), (0, 408197), (832485, 1664970), (0, 1273270), (1787155, 3574310)]
[(833933, 1667867), (0, 1089200), (2749289, 5498578), (0, 2107803), (915069, 1830138), (0, 290038), (408197, 816394), (0, 832485), (1273270, 2546541), (0, 1787155)]


## Loading the FASTA TRAIN and TEST 
together with features tables in dataframes

In [70]:
#!ls 
%cd "/content/drive/My Drive/Model Outputs/genomes"

/content/drive/My Drive/Model Outputs/genomes


### create TRAIN_GENOMES

In [0]:
simplefilter("ignore")
train_genomes = ("0_0_TRAIN_GCF_000008525.1_ASM852v1_genomic.fna",
"1_0_TRAIN_GCF_000008665.1_ASM866v1_genomic.fna",
"2_0_TRAIN_GCF_000008865.2_ASM886v2_genomic.fna",
"3_0_TRAIN_GCF_000009045.1_ASM904v1_genomic.fna",
"4_0_TRAIN_GCF_000027305.1_ASM2730v1_genomic.fna",
"5_0_TRAIN_GCF_000027325.1_ASM2732v1_genomic.fna",
"6_0_TRAIN_GCF_000027345.1_ASM2734v1_genomic.fna",
"7_0_TRAIN_GCF_000091665.1_ASM9166v1_genomic.fna",
"8_0_TRAIN_GCF_000214725.1_ASM21472v1_genomic.fna",
"9_0_TRAIN_GCF_003015225.1_ASM301522v1_genomic.fna")

train_feature_tables = ("GCA_000008525.1_ASM852v1_feature_table.tsv",
"GCA_000008665.1_ASM866v1_feature_table.tsv",
"GCA_000008865.2_ASM886v2_feature_table.tsv",
"GCA_000009045.1_ASM904v1_feature_table.tsv",
"GCA_000027305.1_ASM2730v1_feature_table.tsv",
"GCA_000027325.1_ASM2732v1_feature_table.tsv",
"GCA_000027345.1_ASM2734v1_feature_table.tsv",
"GCA_000091665.1_ASM9166v1_feature_table.tsv",
"GCA_000214725.1_ASM21472v1_feature_table.tsv",
"GCA_003015225.1_ASM301522v1_feature_table.tsv")

### create TEST GENOMES

In [0]:
simplefilter("ignore")
test_genomes = ("0_0_TEST_GCF_000008525.1_ASM852v1_genomic.fna",
"1_0_TEST_GCF_000008665.1_ASM866v1_genomic.fna",
"2_0_TEST_GCF_000008865.2_ASM886v2_genomic.fna",
"3_0_TEST_GCF_000009045.1_ASM904v1_genomic.fna",
"4_0_TEST_GCF_000027305.1_ASM2730v1_genomic.fna",
"5_0_TEST_GCF_000027325.1_ASM2732v1_genomic.fna",
"6_0_TEST_GCF_000027345.1_ASM2734v1_genomic.fna",
"7_0_TEST_GCF_000091665.1_ASM9166v1_genomic.fna",
"8_0_TEST_GCF_000214725.1_ASM21472v1_genomic.fna",
"9_0_TEST_GCF_003015225.1_ASM301522v1_genomic.fna")

test_feature_tables = ("GCA_000008525.1_ASM852v1_feature_table.tsv",
"GCA_000008665.1_ASM866v1_feature_table.tsv",
"GCA_000008865.2_ASM886v2_feature_table.tsv",
"GCA_000009045.1_ASM904v1_feature_table.tsv",
"GCA_000027305.1_ASM2730v1_feature_table.tsv",
"GCA_000027325.1_ASM2732v1_feature_table.tsv",
"GCA_000027345.1_ASM2734v1_feature_table.tsv",
"GCA_000091665.1_ASM9166v1_feature_table.tsv",
"GCA_000214725.1_ASM21472v1_feature_table.tsv",
"GCA_003015225.1_ASM301522v1_feature_table.tsv")

In [74]:
print ("tot genomes: %d; tot feature tables:%d" %(len(train_genomes),len(train_feature_tables))) 
df_train = dict()
for i in range(len(train_feature_tables)):
    df_train[i] = window_pipeline(train_genomes[i], train_feature_tables[i], by = train_splits[i])

tot genomes: 10; tot feature tables:10


In [75]:
## print one example
## "3_0_TEST_GCF_000009045.1_ASM904v1_genomic.fna", (fourth starting counting from 0)
n =4
print (n,df_train[n])
#print (df[n]) # all dataframe
print (type(df_train[n]) )
print (df_train[n].label)#access one colum
print (list(set(df_train[n].label)))# unique entries for one column
#print (df[n][df[n].label== "gene"]) # how to access only genes
print (len(df_train[n][df_train[n].label== "gene"]))

4                                                sequence    label
0     TGGCAATTAAAATTGGTATCAATGGTTTTGGTCGTATCGGCCGTAT...     gene
1     GGTTTAATGACCACTGTTCACGCAACGACTGCAACTCAAAAAACTG...     gene
2     AAACAAAGTATTAGACTTAGTAGCTCATATCTACAACTACAAAGGC...     gene
3     TGAAAATCAAAAATAGGGTTAATATGAATCTCGATCTCCATTTTGT...     gene
4     CATTGGGCAATTTCCGTGGGACAAAAATATTTTGATCTACGTGCGA...     gene
...                                                 ...      ...
2495  CGCACCGAAAATTCGTACGGTTAAATCAGTAGCTAAAAAGATAAAT...  partial
2496  CAATGATGAATATGTGGAAAAAGCTGAGCCAGATGATTGCACGACG...  partial
2497  CGTTTTTCATCAAATTTGAAAGGAGAGAAAATGGAGTTACACAATA...  partial
2498  TTTGTTAAAAGAAGTCAATGAGCAAGGTTTTGTCTTTTTTACCAAT...  partial
2499  AATCCGATAAATATTTTGCTACCCGACCTTACACCAGTCGTATAGG...  partial

[6124 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>
0          gene
1          gene
2          gene
3          gene
4          gene
         ...   
2495    partial
2496    partial
2497    partial
2498    par

In [76]:
print(
    f"columns -> {list(df_train[n].columns)}\n"
    f"labels in dataframe -> {list(df_train[n].label.unique())}"
)

columns -> ['sequence', 'label']
labels in dataframe -> ['gene', 'intergenic', 'partial']


### Deleting lines with lenght = 0

In [77]:
print(df_train[n][df_train[n].sequence.apply(lambda x: len(x)==0)].count())
df_train[n] = df_train[n][~df_train[n].sequence.apply(lambda x: len(x)==0)]

sequence    0
label       0
dtype: int64


We noticed there is always a line with lenght = 0, that we delete from the dataframe.

## One-hot labels encoding

In [78]:
labels = list(df_train[n].label.unique())
lab0 = np.zeros(len(labels))
lab2vec = {} # create empty dictionaries
vec2lab = {}
for i, label in enumerate(list(df_train[n].label.unique())):
    labv = lab0.copy()
    labv[i] = 1
    lab2vec[label] = labv
    vec2lab[tuple(labv)] = label

print(f"lab2vec -> {lab2vec}\nvec2lab -> {vec2lab}")

lab2vec -> {'gene': array([1., 0., 0.]), 'intergenic': array([0., 1., 0.]), 'partial': array([0., 0., 1.])}
vec2lab -> {(1.0, 0.0, 0.0): 'gene', (0.0, 1.0, 0.0): 'intergenic', (0.0, 0.0, 1.0): 'partial'}


## Create train dataframe and test dataframe

## Training dataset

In [79]:
appended_data = []
for i in range(len(df_train)):
  if not i == 5:
    data = df_train[i][df_train[i].label=="gene"].sample(n=300)
    appended_data.append(data)
    data = df_train[i][df_train[i].label=="intergenic"].sample(n=300) #<---- with 350 error, trying to sample without replacement
    appended_data.append(data)
    data = df_train[i][df_train[i].label=="partial"].sample(n=300)
    appended_data.append(data)
  else: 
    data = df_train[i][df_train[i].label=="gene"].sample(n=100)
    appended_data.append(data)
    data = df_train[i][df_train[i].label=="intergenic"].sample(n=100)
    appended_data.append(data)
    data = df_train[i][df_train[i].label=="partial"].sample(n=100)
    appended_data.append(data)

# see pd.concat documentation for more info
training_data = pd.concat(appended_data).reset_index(drop=False)

#shuffle
training_data = training_data.sample(frac=1)
print (training_data)


      index                                           sequence       label
4646    121  AAAAATATCGGTTATTTTTGAATTGATGTTTAGATTTGAACTTCCC...  intergenic
2342    479  GTTTACAGGTATACTCGCTAAAAATTATTCAGCGGGTTTGGAAACA...  intergenic
5952   1829  TATGAAAGTTGATGATATAATTACTTTTGCATTTAAGAATATAAAG...        gene
4541    250  AGAGAAGCTTTGATCCCTGAAAAGATTGTTGGTAAAAATCTGGTTA...        gene
7432   2193  TCATACTTTTCAATCATACAATCACCTATAAATTCCATTCTTAAAA...     partial
...     ...                                                ...         ...
1788   2446  AGCTATAGCAGAGATAGCCGGAGCTCTCCATCCGAGAGCTATGAGA...     partial
7072   1375  TTGTATGGATTTCATGTTTATACAACTTTCCAGTTTTTATAAATTG...  intergenic
2190   1195  CATAGTCGCATGGTGTCGCCTTCTTTACTTTTGTTACTGATTTGTA...  intergenic
6384   1553  CAAAAGGAGTTAAGCCAGCACCAATAATATTTAGACATGATTTTGA...     partial
7559   2250  TATTTCAAGGTAATGGTGTAATCGGGGGAGATTTTGTCAGCAGCGG...        gene

[8400 rows x 3 columns]


## Testing dataset

In [0]:
df_test = dict()
for i in range(len(train_feature_tables)):
    df_train[i] = window_pipeline(train_genomes[i], train_feature_tables[i], by = test_splits[i])

In [58]:

appended_data = []
for i in range(len(df_test)):
  if not i == 5:
    data = df_test[i][df_test[i].label=="gene"]
    data = data[0:300]
    appended_data.append(data)
    data = df_test[i][df_test[i].label=="intergenic"]
    data = data[0:300]
    appended_data.append(data)
    data = df_test[i][df_test[i].label=="partial"]
    data = data[0:300]
    appended_data.append(data)
  else: 
    data = df_test[i][df_test[i].label=="gene"]
    data = data[0:100]
    appended_data.append(data)
    data = df_test[i][df_test[i].label=="intergenic"]
    data = data[0:100]
    appended_data.append(data)
    data = df_test[i][df_test[i].label=="partial"]
    data = data[0:100]
    appended_data.append(data)

# see pd.concat documentation for more info
testing_data = pd.concat(appended_data).reset_index(drop=False)

#no shuffle
#testing_data = testing_data.sample(frac=1)
print (testing_data)


      index                                           sequence    label
0         0  GGCGGTGAAAAAAATCGTTGTGAGTTGGTGTGTGGCGTTGGCTTTT...     gene
1         1  CAAAACACTGAGATTAACAACTTAAGAAAAGTGCAAGAAGTGTTGT...     gene
2         2  AGAGATTATTGCATTAGCTTAAGGCCTTATATCTATAATTGGCGCT...     gene
3         3  GCCATACGCCTTAAGAAAAAGATTTTTCAAACGCTTTGCGCTGATT...     gene
4         4  ATTTTGGGGGGATTGATTATAAAGAGCAATTAAGCATGCTCACGGC...     gene
...     ...                                                ...      ...
8395    295  GGCCAAGTGGAATTTATTTTTGGCCGAAGCCCATTTGCATATTCCC...  partial
8396    296  ATGAACTGGGGCGGGCTGGGGTGGAAGCCGCTCTGAAGGATTTGTC...  partial
8397    297  TTCTACAGATGTGGGTCGGCCTGGCGATCGCCGTGTTTGGTGTTTT...  partial
8398    298  AACTGGCGAATCTTTTGGGAACCCGCTCCCATTTTGTTTTATTTTA...  partial
8399    299  TTGTCTTGAGCGCCATTGTCCCCTCCAGTCCCTACGTGCGGAATGA...  partial

[8400 rows x 3 columns]
